In [11]:
import assemblyai as aai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
print(os.environ.get("ASSEMBLY_API_KEY"))

In [13]:

aai.settings.api_key = os.environ.get("ASSEMBLY_API_KEY")
transcriber = aai.Transcriber()

transcript = transcriber.transcribe("https://karnatakabank.com/sites/default/files/2024-05/tkb0220240524153240_1.mp3")
# transcript = transcriber.transcribe("./my-local-audio-file.wav")


In [32]:
transcript.confidence

0.914397313316422

In [28]:
transcript_chunks = transcript.get_paragraphs()

In [38]:
len(transcript_chunks[7].text)

580

In [14]:
print(transcript.text)

Ladies and gentlemen, good day and welcome to Q four FY 2024 earnings conference call hosted by Karnataka bank. As a reminder, all participant lines will be in the listen only mode and there will be an opportunity for you to ask questions after the presentation concludes. Should you need assistance during the conference call, please signal an operator by pressing zero on a touchstone phone. Please note that this conference is being recorded. I now hand the conference over to Mister Sri Krishnan H, managing director and CEO from Karnataka bank. Thank you and over to you sir. Thank you so much Zika and good evening ladies and gentlemen. This is Sri Krishnan MD, CEO of Karnataka Bank. I am joined here by Shekhar Rao who is our executive director, CFO Abhishek Babchi, COO Balachandra and chief business officer Gokul Das Pai. Welcome to this Karnataka Bank Q four FY 24 annual results and Q four call. We have had an excellent financial year with growth registered across all the key metrics. 

In [18]:
with open('transcripts/transcript.txt','w+') as f :
    f.write(transcript.text)

In [60]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, chat_models
from langchain_community.vectorstores import Pinecone as Pineconelangchain
from pinecone import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import AssemblyAIAudioTranscriptLoader
from langchain_community.document_loaders.assemblyai import TranscriptFormat
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()


True

In [51]:
import assemblyai as aai
config = aai.TranscriptionConfig(punctuate=False,format_text=False)

In [52]:
audio_file = "https://khazanchi.co.in/files2/Khazanchi%20Jewellers%20Limited%20Earning%20Call%20Audio%20-%20H2%20FY24%20(1).mp3"
print("loading transcript...")
loader = AssemblyAIAudioTranscriptLoader(
    file_path=audio_file,config=config)
raw_docs = loader.load()



loading transcript...


In [58]:
type(raw_docs[0])

langchain_core.documents.base.Document

In [53]:
print("splitting")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n","\n"," ",""]
)

documents = text_splitter.split_documents(raw_docs)

print(f"Splitted into {len(documents)} documents")   

splitting
Splitted into 103 documents


In [5]:
embeddings = OpenAIEmbeddings()

In [45]:
#documents_texts  = [{"page_content": d.page_content,"metadata":d.metadata} for d in documents]

In [61]:
documents_texts = [d.page_content for d in documents]

In [62]:
Pineconelangchain.from_texts(documents_texts,embeddings,index_name=os.environ["PINECONE_INDEX_NAME"],)